In [1]:
import pandas as pd

In [2]:
pref = pd.read_csv("COVID-19-jp/nhk_news_covid19_prefectures_daily_data.csv")
pref["日付"] = pd.to_datetime(pref["日付"] )
print(len(pref))
pref.tail()

25521


,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計
25516,2021-07-07,47,沖縄県,58,21165,0,196
25517,2021-07-08,47,沖縄県,35,21200,5,201
25518,2021-07-09,47,沖縄県,55,21255,5,206
25519,2021-07-10,47,沖縄県,64,21319,0,206
25520,2021-07-11,47,沖縄県,30,21349,0,206


In [3]:
latest_date = pref["日付"].max().strftime("%Y-%m-%d")

In [4]:
all_jp = pref.groupby(["都道府県名"], as_index=False).sum()
del all_jp["各地の感染者数_累計"]
del all_jp["各地の死者数_累計"]
#all_jp = all_jp.rename(columns={"各地の感染者数合計":"各地の感染者数_1日ごとの発表数"})
all_jp = all_jp.rename(columns={"各地の感染者数_1日ごとの発表数":"各地の感染者数合計"})
all_jp = all_jp.rename(columns={"各地の死者数_1日ごとの発表数":"各地の死者数合計"})
all_jp.tail()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計
42,静岡県,11946,9598,152
43,香川県,20091,2117,31
44,高知県,21177,1896,27
45,鳥取県,16833,495,2
46,鹿児島県,24978,3736,39


## 都道府県ごとの最新データを付与

In [5]:
# 直近１週間の感染者数
pref["直近1週間の感染者数"] = pref["各地の感染者数_1日ごとの発表数"].rolling(7).sum()
pref.tail(7)

,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数
25514,2021-07-05,47,沖縄県,28,21045,0,191,413.0
25515,2021-07-06,47,沖縄県,62,21107,5,196,391.0
25516,2021-07-07,47,沖縄県,58,21165,0,196,382.0
25517,2021-07-08,47,沖縄県,35,21200,5,201,354.0
25518,2021-07-09,47,沖縄県,55,21255,5,206,348.0
25519,2021-07-10,47,沖縄県,64,21319,0,206,336.0
25520,2021-07-11,47,沖縄県,30,21349,0,206,332.0


### 感染者数前日比

In [6]:
import numpy as np
pref["新規感染者数前日比"] = pref["各地の感染者数_1日ごとの発表数"].pct_change()
# 一番古い日付だけNANを設定
pref.loc[pref["日付"]==pref["日付"].min(),"新規感染者数前日比"] = np.nan
pref["直近1週間の新規感染者数前日比平均"] = pref["新規感染者数前日比"].rolling(7).mean()
pref["新規感染者数先週比"] = pref["各地の感染者数_1日ごとの発表数"].pct_change(periods=7)

pref.tail(10)

,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
25511,2021-07-02,47,沖縄県,61,20907,4,191,435.0,-0.031746,0.171824,-0.256098
25512,2021-07-03,47,沖縄県,76,20983,0,191,437.0,0.245902,0.220891,0.027027
25513,2021-07-04,47,沖縄県,34,21017,0,191,410.0,-0.552632,0.167040,-0.442623
25514,2021-07-05,47,沖縄県,28,21045,0,191,413.0,-0.176471,0.226139,0.120000
25515,2021-07-06,47,沖縄県,62,21107,5,196,391.0,1.214286,0.062465,-0.261905
25516,2021-07-07,47,沖縄県,58,21165,0,196,382.0,-0.064516,0.082160,-0.134328
25517,2021-07-08,47,沖縄県,35,21200,5,201,354.0,-0.396552,0.034039,-0.444444
25518,2021-07-09,47,沖縄県,55,21255,5,206,348.0,0.571429,0.120207,-0.098361
25519,2021-07-10,47,沖縄県,64,21319,0,206,336.0,0.163636,0.108454,-0.157895
25520,2021-07-11,47,沖縄県,30,21349,0,206,332.0,-0.531250,0.111509,-0.117647


In [7]:
latest = pref.loc[pref["日付"]==pref["日付"].max()][["日付","都道府県名","各地の感染者数_1日ごとの発表数","各地の死者数_1日ごとの発表数","直近1週間の感染者数","新規感染者数前日比","直近1週間の新規感染者数前日比平均","新規感染者数先週比"]]
latest.head()

,日付,都道府県名,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
542,2021-07-11,北海道,56,0,362.0,-0.125000,0.110488,0.435897
1085,2021-07-11,青森県,1,0,19.0,inf,NaN,-0.500000
1628,2021-07-11,岩手県,14,0,75.0,0.750000,0.333767,1.333333
2171,2021-07-11,宮城県,21,0,132.0,-0.192308,0.365742,0.750000
2714,2021-07-11,秋田県,4,0,32.0,0.000000,1.087199,0.333333


In [8]:
all_jp = pd.merge(all_jp, latest, on="都道府県名")
all_jp.head()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
0,三重県,13032,5344,113,2021-07-11,17,0,66.0,1.428571,0.485034,3.250000
1,京都府,14118,16813,247,2021-07-11,27,0,154.0,0.125000,0.325527,0.500000
2,佐賀県,22263,2570,24,2021-07-11,0,0,5.0,NaN,NaN,NaN
3,兵庫県,15204,41289,1310,2021-07-11,44,0,259.0,-0.022222,0.203104,0.629630
4,北海道,543,41828,1412,2021-07-11,56,0,362.0,-0.125000,0.110488,0.435897


## 人口データを付与

In [9]:
pop = pd.read_excel("COVID-19-jp/2001stjin.xls",header=3)[["Unnamed: 1","計"]]
pop.rename(columns={"Unnamed: 1":"都道府県名"},inplace=True)
pop.rename(columns={"計":"人口"},inplace=True)
pop = pop.loc[pop["都道府県名"]!="合計"].reset_index(drop=True)

In [10]:
all_jp = pd.merge(all_jp, pop, on="都道府県名", how="left")
all_jp.head()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口
0,三重県,13032,5344,113,2021-07-11,17,0,66.0,1.428571,0.485034,3.250000,1813859
1,京都府,14118,16813,247,2021-07-11,27,0,154.0,0.125000,0.325527,0.500000,2545899
2,佐賀県,22263,2570,24,2021-07-11,0,0,5.0,NaN,NaN,NaN,823810
3,兵庫県,15204,41289,1310,2021-07-11,44,0,259.0,-0.022222,0.203104,0.629630,5549568
4,北海道,543,41828,1412,2021-07-11,56,0,362.0,-0.125000,0.110488,0.435897,5267762


## 人口当たりの感染者数

In [11]:
all_jp["１万人当たりの感染者数"] = all_jp["各地の感染者数合計"] / (all_jp["人口"] / 10000)
all_jp["10万人当たりの感染者数"] = all_jp["各地の感染者数合計"] / (all_jp["人口"] / 100000)
all_jp["直近1週間の10万人当たりの感染者数"] = all_jp["直近1週間の感染者数"] / (all_jp["人口"] / 100000)
all_jp.sort_values("都道府県コード")

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
4,北海道,543,41828,1412,2021-07-11,56,0,362.0,-0.125000,0.110488,0.435897,5267762,79.403739,794.037392,6.871989
41,青森県,1086,2539,31,2021-07-11,1,0,19.0,inf,NaN,-0.500000,1275783,19.901504,199.015036,1.489281
19,岩手県,1629,1788,47,2021-07-11,14,0,75.0,0.750000,0.333767,1.333333,1235517,14.471675,144.716746,6.070333
11,宮城県,2172,9319,91,2021-07-11,21,0,132.0,-0.192308,0.365742,0.750000,2292385,40.651985,406.519847,5.758195
36,秋田県,2715,939,20,2021-07-11,4,0,32.0,0.000000,1.087199,0.333333,985416,9.528971,95.289705,3.247359
15,山形県,3258,2061,47,2021-07-11,4,0,33.0,-0.692308,NaN,3.000000,1082296,19.042850,190.428496,3.049073
35,福島県,3801,5090,161,2021-07-11,16,0,128.0,-0.360000,0.099842,-0.157895,1881981,27.045969,270.459691,6.801344
38,茨城県,4344,10837,166,2021-07-11,35,0,186.0,-0.054054,0.094430,0.166667,2921436,37.094771,370.947712,6.366732
27,栃木県,4887,7242,82,2021-07-11,13,0,122.0,-0.350000,0.099542,0.181818,1965516,36.845286,368.452864,6.207021
37,群馬県,5430,8088,154,2021-07-11,3,0,37.0,-0.785714,0.716327,0.000000,1969439,41.067532,410.675324,1.878708


## 感染者数

### 平均値、中央値

In [12]:
all_jp["各地の感染者数合計"].describe()

count        47.000000
mean      17386.638298
std       32303.305571
min         495.000000
25%        2554.500000
50%        5090.000000
75%       11206.000000
max      181638.000000
Name: 各地の感染者数合計, dtype: float64

### TOP 10

In [13]:
all_jp.sort_values("各地の感染者数合計", ascending=False).reset_index(drop=True).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
0,東京都,7059,181638,2256,2021-07-11,614,3,5137.0,-0.353684,0.091277,0.185328,13834925,131.289472,1312.894721,37.130667
1,大阪府,14661,104849,2693,2021-07-11,167,0,999.0,-0.165000,0.136706,0.897727,8849635,118.478333,1184.783327,11.288601
2,神奈川県,7602,70315,964,2021-07-11,389,0,2004.0,0.254839,0.096807,0.721239,9209442,76.350988,763.509885,21.760276
3,愛知県,12489,51696,983,2021-07-11,64,1,401.0,0.000000,0.190291,0.000000,7575530,68.240770,682.407700,5.293359
4,埼玉県,5973,47925,838,2021-07-11,163,1,985.0,0.108844,0.086481,0.314516,7390054,64.850676,648.506763,13.328725
5,千葉県,6516,42029,717,2021-07-11,183,1,1155.0,-0.102941,0.057755,0.297872,6319772,66.503981,665.039815,18.275976
6,北海道,543,41828,1412,2021-07-11,56,0,362.0,-0.125000,0.110488,0.435897,5267762,79.403739,794.037392,6.871989
7,兵庫県,15204,41289,1310,2021-07-11,44,0,259.0,-0.022222,0.203104,0.629630,5549568,74.400386,744.003858,4.667030
8,福岡県,21720,35930,525,2021-07-11,35,0,292.0,-0.351852,0.174854,0.250000,5129841,70.041157,700.411572,5.692184
9,沖縄県,25521,21349,206,2021-07-11,30,0,332.0,-0.531250,0.111509,-0.117647,1481547,144.099377,1440.993772,22.409009


## １万人あたりの感染者数

### 平均値、中央値

In [14]:
all_jp["１万人当たりの感染者数"].describe()

count     47.000000
mean      41.708230
std       30.510546
min        8.258210
25%       22.677876
50%       30.437037
75%       53.671054
max      144.099377
Name: １万人当たりの感染者数, dtype: float64

## 直近１週間の１０万人あたりの感染者数

In [15]:
all_jp.sort_values(["直近1週間の10万人当たりの感染者数"],ascending=False).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
26,東京都,7059,181638,2256,2021-07-11,614,3,5137.0,-0.353684,0.091277,0.185328,13834925,131.289472,1312.894721,37.130667
28,沖縄県,25521,21349,206,2021-07-11,30,0,332.0,-0.531250,0.111509,-0.117647,1481547,144.099377,1440.993772,22.409009
32,神奈川県,7602,70315,964,2021-07-11,389,0,2004.0,0.254839,0.096807,0.721239,9209442,76.350988,763.509885,21.760276
5,千葉県,6516,42029,717,2021-07-11,183,1,1155.0,-0.102941,0.057755,0.297872,6319772,66.503981,665.039815,18.275976
7,埼玉県,5973,47925,838,2021-07-11,163,1,985.0,0.108844,0.086481,0.314516,7390054,64.850676,648.506763,13.328725
9,大阪府,14661,104849,2693,2021-07-11,167,0,999.0,-0.165000,0.136706,0.897727,8849635,118.478333,1184.783327,11.288601
33,福井県,9774,1389,35,2021-07-11,16,0,62.0,-0.272727,0.675325,4.333333,780053,17.806482,178.064824,7.948178
31,石川県,9231,4034,115,2021-07-11,11,0,80.0,-0.266667,0.206887,1.200000,1139612,35.398013,353.980127,7.019933
4,北海道,543,41828,1412,2021-07-11,56,0,362.0,-0.125000,0.110488,0.435897,5267762,79.403739,794.037392,6.871989
35,福島県,3801,5090,161,2021-07-11,16,0,128.0,-0.360000,0.099842,-0.157895,1881981,27.045969,270.459691,6.801344


## 直近１週間の新規感染者 増加速度

## 増加速度 統計情報

In [16]:
all_jp["直近1週間の新規感染者数前日比平均"].describe()

count    29.000000
mean      0.306752
std       0.289854
min      -0.009524
25%       0.099542
50%       0.190291
75%       0.368121
max       1.087199
Name: 直近1週間の新規感染者数前日比平均, dtype: float64

### WORST 10(増加速度の早い順)

In [17]:
all_jp.sort_values("直近1週間の新規感染者数前日比平均",ascending=False).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
36,秋田県,2715,939,20,2021-07-11,4,0,32.0,0.000000,1.087199,0.333333,985416,9.528971,95.289705,3.247359
46,鹿児島県,24978,3736,39,2021-07-11,5,0,38.0,0.250000,0.888683,0.666667,1630146,22.918193,229.181926,2.331080
44,高知県,21177,1896,27,2021-07-11,7,0,45.0,-0.222222,0.840420,0.000000,709230,26.733218,267.332177,6.344909
37,群馬県,5430,8088,154,2021-07-11,3,0,37.0,-0.785714,0.716327,0.000000,1969439,41.067532,410.675324,1.878708
33,福井県,9774,1389,35,2021-07-11,16,0,62.0,-0.272727,0.675325,4.333333,780053,17.806482,178.064824,7.948178
14,山口県,19005,3175,80,2021-07-11,2,0,16.0,0.000000,0.619048,0.000000,1369882,23.177179,231.771788,1.167984
0,三重県,13032,5344,113,2021-07-11,17,0,66.0,1.428571,0.485034,3.250000,1813859,29.462047,294.620475,3.638651
10,奈良県,15747,8333,132,2021-07-11,9,0,87.0,-0.590909,0.368121,0.800000,1353837,61.550984,615.509844,6.426180
11,宮城県,2172,9319,91,2021-07-11,21,0,132.0,-0.192308,0.365742,0.750000,2292385,40.651985,406.519847,5.758195
19,岩手県,1629,1788,47,2021-07-11,14,0,75.0,0.750000,0.333767,1.333333,1235517,14.471675,144.716746,6.070333


### BEST 10(増加速度の遅い順／減少速度の速い順)

In [18]:
all_jp.sort_values("直近1週間の新規感染者数前日比平均",ascending=True).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
17,岐阜県,11403,9307,187,2021-07-11,3,0,25.0,-0.400000,-0.009524,-0.625000,2032490,45.791123,457.911232,1.230018
5,千葉県,6516,42029,717,2021-07-11,183,1,1155.0,-0.102941,0.057755,0.297872,6319772,66.503981,665.039815,18.275976
7,埼玉県,5973,47925,838,2021-07-11,163,1,985.0,0.108844,0.086481,0.314516,7390054,64.850676,648.506763,13.328725
26,東京都,7059,181638,2256,2021-07-11,614,3,5137.0,-0.353684,0.091277,0.185328,13834925,131.289472,1312.894721,37.130667
38,茨城県,4344,10837,166,2021-07-11,35,0,186.0,-0.054054,0.094430,0.166667,2921436,37.094771,370.947712,6.366732
32,神奈川県,7602,70315,964,2021-07-11,389,0,2004.0,0.254839,0.096807,0.721239,9209442,76.350988,763.509885,21.760276
21,広島県,18462,11575,176,2021-07-11,8,0,50.0,0.600000,0.097506,0.142857,2826858,40.946521,409.465208,1.768748
27,栃木県,4887,7242,82,2021-07-11,13,0,122.0,-0.350000,0.099542,0.181818,1965516,36.845286,368.452864,6.207021
35,福島県,3801,5090,161,2021-07-11,16,0,128.0,-0.360000,0.099842,-0.157895,1881981,27.045969,270.459691,6.801344
42,静岡県,11946,9598,152,2021-07-11,13,0,197.0,-0.648649,0.103997,0.000000,3708556,25.880693,258.806932,5.312041


## グラフ化

In [19]:
import matplotlib.pyplot as plt
%matplotlib notebook
#%matplotlib inline

In [20]:
all_jp_desc = all_jp.sort_values(["各地の感染者数合計"],ascending=False)
all_jp_desc_10k = all_jp.sort_values(["１万人当たりの感染者数"],ascending=False)
all_jp_desc_100k = all_jp.sort_values(["直近1週間の10万人当たりの感染者数"],ascending=False)

In [21]:
fig = plt.figure(figsize=(9.5,5))
plt.subplot(1,2,1) # (rows, columns, panel number)
plt.title("各地の新規感染者数累計")
plt.yticks(fontsize=8)
plt.grid(axis="x")
chart_data = all_jp.sort_values(["各地の感染者数合計"])
plt.barh(chart_data["都道府県名"],chart_data["各地の感染者数合計"],align="center")

plt.subplot(1,2,2) # (rows, columns, panel number)
plt.title("1万人当たりの新規感染者数累計")
plt.yticks(fontsize=8)
plt.grid(axis="x")
chart_data = all_jp.sort_values(["１万人当たりの感染者数"])
plt.barh(chart_data["都道府県名"], chart_data["１万人当たりの感染者数"])

fig.text(0,0,"※新型コロナ関連の情報提供:NHK ※人口の情報提供:e-Stat",fontsize=8)
fig.text(0,0.98,"※{}時点".format(latest_date),fontsize=9,va="top")

fig.tight_layout()

<IPython.core.display.Javascript object>

In [22]:
fig = plt.figure(figsize=(9.5,10))

#plt.style.use("ggplot")
### 最新の各地の新規感染者数
plt.subplot(2,2,1) # (rows, columns, panel number)
plt.title("{}の各地の新規感染者数".format(latest_date))
chart_data = all_jp.sort_values(["各地の感染者数_1日ごとの発表数"])
plt.barh(chart_data["都道府県名"], chart_data["各地の感染者数_1日ごとの発表数"])
plt.yticks(fontsize=8)
plt.grid(axis="x")

### 最新の各地の死者数
plt.subplot(2,2,2) # (rows, columns, panel number)
plt.title("{}の各地の新規死者数".format(latest_date))
chart_data = all_jp.sort_values(["各地の死者数_1日ごとの発表数"])
plt.barh(chart_data["都道府県名"], chart_data["各地の死者数_1日ごとの発表数"])
plt.yticks(fontsize=8)
plt.grid(axis="x")



### 直近1週間の10万人当たりの新規感染者数
plt.subplot(2,2,3) # (rows, columns, panel number)
plt.title("直近1週間の10万人当たりの新規感染者数")
chart_data = all_jp.sort_values(["直近1週間の10万人当たりの感染者数"])
plt.barh(chart_data["都道府県名"], chart_data["直近1週間の10万人当たりの感染者数"])
plt.plot([15,15],[0,47],"--",lw=0.5,color="black")
plt.text(15,0,"Stage3",ha="center",va="top",fontsize=8)
plt.plot([25,25],[0,47],"--",lw=0.5,color="black")
plt.text(25,0,"Stage4",ha="center",va="top",fontsize=8)
plt.yticks(fontsize=8)
plt.grid(axis="x")

### 新規感染者数の増加速度（新規感染者数前週同曜日比）
plt.subplot(2,2,4) # (rows, columns, panel number)
plt.title("新規感染者数前週同曜日比")
chart_data = all_jp.sort_values("新規感染者数先週比",na_position="first")
plt.barh(chart_data["都道府県名"], chart_data["新規感染者数先週比"])
plt.yticks(fontsize=8)
plt.grid(axis="x")

fig.text(0,0,"※新型コロナ関連の情報提供:NHK ※人口の情報提供:e-Stat",fontsize=8)
fig.text(0,0.98,"※{}時点".format(latest_date),fontsize=9,va="top")

fig.tight_layout()

<IPython.core.display.Javascript object>

In [23]:
fig.savefig("out/covid-domestic.png")

## 度数分布表

In [24]:
import numpy as np
data = np.array(all_jp["各地の感染者数合計"])
 
# ヒストグラム
hist, bin_edges = np.histogram(data, bins=8)

print(hist)
print(bin_edges)

[38  4  2  1  1  0  0  1]
[   495.     23137.875  45780.75   68423.625  91066.5   113709.375
 136352.25  158995.125 181638.   ]


## ヒストグラム

In [25]:
fig = plt.figure(figsize=(10,3))
plt.hist(all_jp_desc["各地の感染者数合計"],bins=[0,20000,40000,80000,100000,120000,140000,160000])

<IPython.core.display.Javascript object>

(array([37.,  2.,  6.,  0.,  1.,  0.,  0.]),
 array([     0,  20000,  40000,  80000, 100000, 120000, 140000, 160000]),
 <a list of 7 Patch objects>)

In [26]:
fig = plt.figure(figsize=(10,3))
plt.hist(all_jp_desc["10万人当たりの感染者数"],bins=[0,200,400,600,800,1000,1200])


<IPython.core.display.Javascript object>

(array([ 9., 21.,  5.,  9.,  0.,  1.]),
 array([   0,  200,  400,  600,  800, 1000, 1200]),
 <a list of 6 Patch objects>)

In [27]:
pref[pref["都道府県名"]=="山口県"].tail(10)


,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
18995,2021-07-02,35,山口県,7,3152,0,77,27.0,0.750000,NaN,0.166667
18996,2021-07-03,35,山口県,5,3157,1,78,27.0,-0.285714,NaN,0.000000
18997,2021-07-04,35,山口県,2,3159,2,80,27.0,-0.600000,NaN,0.000000
18998,2021-07-05,35,山口県,1,3160,0,80,27.0,-0.500000,NaN,0.000000
18999,2021-07-06,35,山口県,2,3162,0,80,29.0,1.000000,NaN,inf
19000,2021-07-07,35,山口県,1,3163,0,80,22.0,-0.500000,-0.090816,-0.875000
19001,2021-07-08,35,山口県,6,3169,0,80,24.0,5.000000,0.694898,0.500000
19002,2021-07-09,35,山口県,2,3171,0,80,19.0,-0.666667,0.492517,-0.714286
19003,2021-07-10,35,山口県,2,3173,0,80,16.0,0.000000,0.533333,-0.600000
19004,2021-07-11,35,山口県,2,3175,0,80,16.0,0.000000,0.619048,0.000000
